In [ ]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user() 
project_id = "sb-metrics"

In [ ]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [ ]:
# Add imports for any libraries 
import pandas as pd
import datetime
import pytz
import gspread

from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
from gspread_dataframe import set_with_dataframe

tz = pytz.timezone('US/Pacific')

In [ ]:
# set up for saving dataframes to google sheets
time = datetime.datetime.now(tz).strftime("%m/%d/%y %H:%M")
report_title = 'CDP STP Users Report'
report_sheet = gc.open(report_title)

views_title = 'Users Report Views'
views_sheet = gc.open(views_title)

In [ ]:
params = {"from_date": '2021-01-01', "to_date": '2021-01-31', "org_id": 2}


The following query pulls from views in BigQuery:

*   `sb-metrics.merged_views.all_project_users`
    * View that connects memberships to projects categorized by user license type 
*   `sb-metrics.merged_views.memberships_users_teams_roles`
    * View that displays all memberships  
*   `sb-metrics.merged_views.solicitations_purchasing_agency`
    * View that displays all projects 

The query joins the three tables and filters by three variables:


*   org_id: team_id of interest
*   from_date: beginning date in period of interest
*   to_date: ending date in period of interest

The query finds all active users who are part of solicitations (belonging to the organization of interest) that were live during period of interest.

If you want the live solicitations right now, set both the from_date and to_date to a date in the future.



In [ ]:
%%bigquery --project $project_id report --params $params 

SELECT
  users.user_email,
  users.user_first_name,
  users.user_last_name,
  users.user_type,
  solicitations.proj_name AS solicitation_name,
  solicitations.purchasing_agency,
  users.invite_created_at,
  users.user_created_at,
  users.accepted_invite,
  solicitations.proj_created_at,
  solicitations.proj_updated_at,
  solicitations.proj_published_at,
  solicitations.proj_archived_at,
  users.membership_role
FROM
  `sb-metrics.merged_views.solicitations_purchasing_agency` AS solicitations
LEFT JOIN (
  SELECT
    users_license_type.project_id,
    users_license_type.membership_id,
    memberships.user_email,
    memberships.user_first_name,
    memberships.user_last_name,
    users_license_type.user_type,
    users_license_type.invite_created_at,
    memberships.user_id,
    memberships.user_created_at,
    memberships.accepted_invite,
    memberships.membership_role
  FROM
    `sb-metrics.merged_views.memberships_users_teams_roles` AS memberships
  LEFT JOIN
    `sb-metrics.merged_views.all_project_users` AS users_license_type
  ON
    memberships.membership_id = users_license_type.membership_id) users
ON
  solicitations.proj_id = users.project_id
WHERE
  solicitations.proj_team_id = @org_id
  AND solicitations.proj_created_at < @to_date
  AND 
  ((solicitations.proj_updated_at < @to_date AND solicitations.proj_updated_at > @from_date)
  OR (
    (solicitations.proj_published_at > @from_date OR solicitations.proj_published_at IS NULL) 
    AND (solicitations.proj_archived_at > @from_date OR solicitations.proj_archived_at IS NULL)
    )
  )
  AND users.invite_created_at < @to_date
  AND users.accepted_invite IS TRUE
ORDER BY
  user_first_name

In [ ]:
report

,user_email,user_first_name,user_last_name,user_type,solicitation_name,purchasing_agency,invite_created_at,user_created_at,accepted_invite,proj_created_at,proj_updated_at,proj_published_at,proj_archived_at,membership_role
0,Annette.Stanfield@edd.ca.gov,Annette,Stanfield,contributors,Document Management System Modernization (DMSM),Employment Development Department,2021-01-28,2021-01-28,True,2021-01-23,2021-02-01,None,None,None
1,Athena.Pennington@state.ca.gov,Athena,Pennington,contributors,CCVS,California Department of Public Health,2020-09-09,2020-08-13,True,2020-09-08,2021-02-03,None,None,staff
2,Becky.Fatur@state.ca.gov,Becky,Fatur,editors,"SURVEILLANCE, HEALTH, INTERVENTION, ENVIRONMEN...",California Department of Public Health,2021-01-27,2020-04-30,True,2021-01-26,2021-02-03,None,None,admin
3,Becky.Fatur@state.ca.gov,Becky,Fatur,owners,California State Payroll System (CSPS) (OLD ON...,State Controller's Office,2020-11-11,2020-04-30,True,2020-11-11,2021-02-02,None,None,admin
4,Becky.Fatur@state.ca.gov,Becky,Fatur,owners,California State Payroll System (CSPS),State Controller's Office,2020-12-15,2020-04-30,True,2020-12-15,2021-02-03,None,None,admin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Wanda.Yanez@edd.ca.gov,Wanda,Yanez,contributors,Document Management System Modernization (DMSM),Employment Development Department,2021-01-28,2021-01-28,True,2021-01-23,2021-02-01,None,None,None
94,YCasey@sco.ca.gov,Yvette,Casey,contributors,California State Payroll System (CSPS),State Controller's Office,2020-12-18,2021-01-07,True,2020-12-15,2021-02-03,None,None,None
95,melissa.thompson@dot.ca.gov,melissa,thompson,contributors,Transportation Asset Management System (TAMS),Department of Transportation,2021-01-21,2021-01-28,True,2021-01-19,2021-02-03,None,None,None
96,nicole.McClain@dot.ca.gov,nicole,McClain,contributors,Transportation Asset Management System (TAMS),Department of Transportation,2021-01-26,2021-01-26,True,2021-01-19,2021-02-03,None,None,None


In [ ]:
# create, and save report
worksheet = report_sheet.add_worksheet(title=time, rows="1000", cols="20")
set_with_dataframe(worksheet, report) 

**The following is the code for the views**

The following is the query that creates the `sb-metrics.merged_views.projects_owners` view - it feeds into the query for creating the `sb-metrics.merged_views.all_project_users` view.

In [ ]:
%%bigquery --project $project_id projects_owners

SELECT
  id AS project_id,
  procurement_official_id AS membership_id,
  EXTRACT(DATE FROM created_at) AS invite_created_at,
  'owners' AS user_type
FROM
  `sb-metrics.postgres_rds_public.projects`
WHERE
  id IN (SELECT proj_id FROM `sb-metrics.merged_views.solicitations_purchasing_agency`)


In [ ]:
projects_owners

,project_id,membership_id,invite_created_at,user_type
0,532,3,2020-08-16,owners
1,540,4,2020-08-21,owners
2,517,5,2020-08-13,owners
3,539,5,2020-08-20,owners
4,542,5,2020-08-24,owners
5,543,5,2020-08-24,owners
6,575,5,2020-09-23,owners
7,643,9,2020-12-09,owners
8,618,268,2020-11-02,owners
9,646,268,2020-12-11,owners


The following is the query that creates the `sb-metrics.merged_views.all_project_users` view





In [ ]:
%%bigquery --project $project_id all_project_users

 (
  SELECT
    *
  FROM
    `sb-metrics.merged_views.projects_owners`)
UNION ALL (
  SELECT
    project_id,
    membership_id,
    EXTRACT(DATE FROM created_at) AS invite_created_at,
    'viewers' AS user_type
  FROM
    `sb-metrics.postgres_rds_public.projects_viewers`)
UNION ALL (
  SELECT
    project_id,
    membership_id,
    EXTRACT(DATE FROM created_at) AS invite_created_at,
    'editors' AS user_type
  FROM
    `sb-metrics.postgres_rds_public.projects_editors`)
UNION ALL (
  SELECT
    project_id,
    membership_id,
    EXTRACT(DATE FROM created_at) AS invite_created_at,
    'contributors' AS user_type
  FROM
    `sb-metrics.postgres_rds_public.projects_contributors`)

In [ ]:
all_project_users

,project_id,membership_id,invite_created_at,user_type
0,517,3,2020-08-13,contributors
1,519,3,2020-08-14,contributors
2,524,5,2020-08-16,contributors
3,539,3,2020-08-20,contributors
4,539,173,2020-08-20,contributors
...,...,...,...,...
309,624,228,2020-11-11,owners
310,600,229,2020-10-05,owners
311,700,229,2021-01-13,owners
312,622,230,2020-11-09,owners


In [ ]:
title = "proj_users " + time
worksheet = views_sheet.add_worksheet(title=title, rows="1000", cols="20")
set_with_dataframe(worksheet, all_project_users) 

The following is the query that creates the `sb-metrics.merged_views.memberships_users_teams_roles` view

In [ ]:
%%bigquery --project $project_id memberships_users_teams_roles

SELECT
  memberships.id AS membership_id,
  memberships.user_id,
  memberships.team_id,
  teams.name AS team_name,
  memberships.user_first_name,
  memberships.user_last_name,
  memberships.user_email,
  (CASE
      WHEN memberships.last_seen_at IS NULL THEN FALSE
      ELSE TRUE
  END) AS accepted_invite,
  EXTRACT(DATE FROM users.created_at) AS user_created_at,
  named_roles.key AS membership_role
FROM
  `sb-metrics.postgres_rds_public.memberships` AS memberships
LEFT JOIN
  `sb-metrics.postgres_rds_public.users` AS users
ON
  memberships.user_id = users.id
LEFT JOIN
  `sb-metrics.postgres_rds_public.teams` AS teams
ON
  memberships.team_id = teams.id
LEFT JOIN (
  SELECT
    membership_roles.membership_id,
    roles.key
  FROM
    `sb-metrics.postgres_rds_public.membership_roles` AS membership_roles
  LEFT JOIN
    `sb-metrics.postgres_rds_public.roles` AS roles
  ON
    membership_roles.role_id = roles.id ) named_roles
ON
  memberships.id = named_roles.membership_id
WHERE
  (memberships.user_first_name IS NOT NULL
    AND memberships.user_last_name IS NOT NULL)
  AND users.former_user IS NOT TRUE
  AND LOWER(memberships.user_email) NOT LIKE '%@cityinnovate%'
  AND LOWER(teams.name) NOT LIKE '%test%'

In [ ]:
memberships_users_teams_roles

,membership_id,user_id,team_id,team_name,user_first_name,user_last_name,user_email,accepted_invite,user_created_at,membership_role
0,267,169.0,2,California Department of Technology - STP,Melissa,Stancell,Melissa.Stancell@dmv.ca.gov,True,2020-10-27,staff
1,266,167.0,2,California Department of Technology - STP,Rafael,Gonzalez,Rafael.Gonzalez@dmv.ca.gov,True,2020-10-26,None
2,264,NaN,2,California Department of Technology - STP,Pam,Fuller,Pam.Fuller@ftb.ca.gov,False,None,None
3,280,NaN,2,California Department of Technology - STP,Dale,Montoya,Dale.montoya@dot.ca.gov,False,None,None
4,277,175.0,43,California Department of Public Health,Maggie,Jimenez,Maggie.Jimenez@cdph.ca.gov,True,2020-11-02,None
...,...,...,...,...,...,...,...,...,...,...
186,300,NaN,43,California Department of Public Health,Theresa,Jones,Theresa.Jones@state.ca.gov,False,None,admin
187,300,NaN,43,California Department of Public Health,Theresa,Jones,Theresa.Jones@state.ca.gov,False,None,staff
188,288,80.0,43,California Department of Public Health,Becky,Fatur,becky.Fatur@state.ca.gov,True,2020-04-30,staff
189,299,184.0,43,California Department of Public Health,Marlon,Paulo,Marlon.Paulo@state.ca.gov,True,2020-12-16,staff


In [ ]:
title = "memberships_users " + time
worksheet = views_sheet.add_worksheet(title=title, rows="1000", cols="20")
set_with_dataframe(worksheet, memberships_users_teams_roles) 

The following is the query that creates the `sb-metrics.merged_views.solicitations_purchasing_agency` view

In [ ]:
%%bigquery --project $project_id solicitations_purchasing_agency

SELECT
  projects.id AS proj_id,
  projects.team_id AS proj_team_id,
  projects.name AS proj_name,
  projects.purchasing_agency_id,
  agencies.name AS purchasing_agency,
  EXTRACT(DATE FROM projects.updated_at) AS proj_updated_at,  
  EXTRACT(DATE FROM projects.created_at) AS proj_created_at,
  EXTRACT(DATE FROM projects.published_at) AS proj_published_at,
  EXTRACT(DATE FROM projects.archived_at) AS proj_archived_at
FROM
  `sb-metrics.postgres_rds_public.projects` AS projects
LEFT JOIN `sb-metrics.postgres_rds_public.purchasing_agencies`AS agencies ON projects.purchasing_agency_id = agencies.id
WHERE 
  LOWER(projects.name) NOT LIKE '%test%'
ORDER BY proj_created_at


In [ ]:
solicitations_purchasing_agency

,proj_id,proj_team_id,proj_name,purchasing_agency_id,purchasing_agency,proj_updated_at,proj_created_at,proj_published_at,proj_archived_at
0,511,2,Emergency Solicitation (BlueDot data entry),NaN,None,2021-02-02,2020-08-09,None,None
1,517,2,Contract for CDPH Challenge,NaN,None,2021-02-02,2020-08-13,None,2020-08-16
2,520,2,California COVID Reporting System (CCRS) OLD,1.0,California Department of Public Health,2021-02-03,2020-08-15,None,2020-09-03
3,532,2,California COVID Reporting System (CCRS) 8.16,NaN,None,2021-02-02,2020-08-16,None,2020-08-16
4,521,2,California COVID Reporting System (CCRS),1.0,California Department of Public Health,2021-02-03,2020-08-16,None,2020-08-16
5,522,2,California COVID Reporting System (CCRS),1.0,California Department of Public Health,2021-02-03,2020-08-16,None,2020-08-16
6,529,2,California Covid Reporting System (CCRS),1.0,California Department of Public Health,2021-02-03,2020-08-16,None,2020-08-16
7,530,2,California COVID Reporting System (CCRS),NaN,None,2021-02-03,2020-08-16,None,2020-08-16
8,533,2,California COVID Reporting System (CCRS) 8.16 ...,NaN,None,2021-02-03,2020-08-16,None,2020-08-16
9,534,2,California COVID Reporting System (CCRS),1.0,California Department of Public Health,2021-02-03,2020-08-16,None,2020-08-18


In [ ]:
title = "solicitations " + time
worksheet = views_sheet.add_worksheet(title=title, rows="1000", cols="20")
set_with_dataframe(worksheet, solicitations_purchasing_agency) 